# Khai báo thư viện và lấy data từ MNIST

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from skimage.transform import rescale, resize, downscale_local_mean

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (48, 48, 1)
# Load the data and split it between train and test sets
#(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
(new_x_train, y_train), (new_x_test, y_test) = keras.datasets.mnist.load_data()

# Phần chuyển ảnh từ 28x28 sang 224x224


In [ ]:
length = 10000
def convert(original):
  def generator():
    for img in range(length):
      i = cv2.resize(original[img], (48,48))
      yield i
  x = np.ndarray(shape=(length,48,48), dtype=np.float16)
  cnt = 0
  for i in generator():
    x[cnt] = i
    cnt += 1
  return x
x_train = convert(new_x_train)
y_train = y_train[:length]
x_test = convert(new_x_test)
y_test = y_test[:length]

# Phần tiền xử lí ảnh

In [ ]:
'''
# Scale images to the [0, 1] range
x_train = x_train.astype("float16") / 255
x_test = x_test.astype("float16") / 255
'''

'\n# Scale images to the [0, 1] range\nx_train = x_train.astype("float16") / 255\nx_test = x_test.astype("float16") / 255\n'

In [ ]:
# Make sure images have shape (28, 28, 1)

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [ ]:
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (10000, 48, 48, 1)
10000 train samples
10000 test samples


# Xây dựng model

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding='same'),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)),
     
        layers.Conv2D(128, kernel_size=(3, 3), activation="relu", padding='same'),
        layers.Conv2D(128, kernel_size=(3, 3), activation="relu", padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)),
     
        layers.Conv2D(256, kernel_size=(3, 3), activation="relu", padding='same'),
        layers.Conv2D(256, kernel_size=(3, 3), activation="relu", padding='same'),
        layers.Conv2D(256, kernel_size=(3, 3), activation="relu", padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)),
     
        layers.Conv2D(512, kernel_size=(3, 3), activation="relu", padding='same'),
        layers.Conv2D(512, kernel_size=(3, 3), activation="relu", padding='same'),
        layers.Conv2D(512, kernel_size=(3, 3), activation="relu", padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)),
     
        layers.Conv2D(512, kernel_size=(3, 3), activation="relu", padding='same'),
        layers.Conv2D(512, kernel_size=(3, 3), activation="relu", padding='same'),
        layers.Conv2D(512, kernel_size=(3, 3), activation="relu", padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)),

        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_143 (Conv2D)         (None, 48, 48, 64)        640       
                                                                 
 conv2d_144 (Conv2D)         (None, 48, 48, 64)        36928     
                                                                 
 max_pooling2d_55 (MaxPoolin  (None, 24, 24, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_145 (Conv2D)         (None, 24, 24, 128)       73856     
                                                                 
 conv2d_146 (Conv2D)         (None, 24, 24, 128)       147584    
                                                                 
 max_pooling2d_56 (MaxPoolin  (None, 12, 12, 128)      0         
 g2D)                                                

# Train model

In [ ]:
batch_size = 128
epochs = 10

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/10
71/71 [==============================] - 8s 104ms/step - loss: 2.3398 - accuracy: 0.1151 - val_loss: 2.3020 - val_accuracy: 0.1040
Epoch 2/10
71/71 [==============================] - 7s 100ms/step - loss: 2.1014 - accuracy: 0.2349 - val_loss: 1.4053 - val_accuracy: 0.5530
Epoch 3/10
71/71 [==============================] - 7s 101ms/step - loss: 0.7551 - accuracy: 0.7618 - val_loss: 0.6027 - val_accuracy: 0.8750
Epoch 4/10
71/71 [==============================] - 7s 101ms/step - loss: 0.2675 - accuracy: 0.9223 - val_loss: 0.2238 - val_accuracy: 0.9300
Epoch 5/10
71/71 [==============================] - 7s 101ms/step - loss: 0.1497 - accuracy: 0.9571 - val_loss: 0.1246 - val_accuracy: 0.9620
Epoch 6/10
71/71 [==============================] - 7s 102ms/step - loss: 0.1213 - accuracy: 0.9656 - val_loss: 0.0929 - val_accuracy: 0.9710
Epoch 7/10
71/71 [==============================] - 7s 102ms/step - loss: 0.0848 - accuracy: 0.9754 - val_loss: 0.0994 - val_accuracy: 0.9680
Epoch 

# Evaluate model

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.1442437618970871
Test accuracy: 0.9675999879837036


# Summary

***TEST 1*** \\
Với model VGG bỏ đi 7 lớp Convo đầu, input 28*28 train mất gần 1p40s cho mỗi epoch và cho Test accuracy là 0.9903, test loss là 0.065. (epoch=30)

***TEST 2*** \\
Với model VGG giữ nguyên và bỏ đi 3 lớp pool đầu (và do đó 7 lớp convo đầu có 
```
padding = 'same'
```
để giữ nguyên size. \\
Kết quả với epoch = 10 như sau: \\
Test loss: 0.022525882348418236 \\
Test accuracy: 0.9927999973297119